# Scryfall API ingestion for T2 Cards

## Imports

In [0]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *

## Ingestion

In [0]:
url = "https://api.scryfall.com/cards/search"

params = {"q": "format:standard", "order": "color", "unique": "cards", "dir": "asc"}

cards = []
while url:
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    cards.extend(data['data'])
    
    url = data.get('next_page', None)
    
    params = None

df_cards = pd.json_normalize(cards, sep='_')

cols = ['name', 'mana_cost', 'type_line', 'oracle_text', 'power', 'toughness', 'loyalty', 'colors', 'color_identity', 'set', 'set_name', 'scryfall_uri']
df_cards = df_cards[cols]

for col in ['colors', 'color_identity']:
    df_cards[col] = df_cards[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else '')

df_cards.display()
print(f"{len(df_cards)} cartas encontradas no Standard")


## Saving df in Unity Catalog

In [0]:
spark_df = spark.createDataFrame(df_cards)

spark_df.write.format("delta").mode("overwrite").saveAsTable("mtg.bronze.t2_cards_ingestion")

print(f"Tabela salva com sucesso em: mtg.bronze")
